In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import time

# Data processing

In [ ]:
data_str = "oilgas-10y"
S_df = pd.read_csv("C:\\Users\\zhubr\\Desktop\\STAT 491\\data-"+data_str+".csv")
S_df = S_df.drop(columns=['Date'])
S_df = S_df.iloc[::-1].astype('float')
S_df.head()

In [ ]:
S_norm_df = (S_df-S_df.mean())/S_df.std()
S_norm_df

# AR(p) model

In [ ]:
class arfit_output():
    def __init__(self,c,ϕ,σ,w,l):
        self.c = c
        self.ϕ = ϕ
        self.σ = σ
        self.w = w
        if (np.sum(ϕ)==1):
            self.μ = c/1e-8
        else:
            self.μ = c/(1-np.sum(ϕ))
        self.l = l
        
class arfit_eval():
    def __init__(self,loglik,aic,bic):
        self.loglik = loglik
        self.aic = aic
        self.bic = bic
        
def likelihood(p,S,arfit):
    c = arfit.c
    ϕ = arfit.ϕ
    σ = arfit.σ
    w = arfit.w
    v = S@w
    T = len(v)
    loglik = 0
    for t in range(p,T):
        loglik = loglik-(v[t]-c-ϕ@np.flip(v[(t-p):t]))**2
    loglik = loglik/(2*σ*σ)-0.5*(T-p)*(np.log(2*np.pi)+np.log(σ*σ))
    aic = -2*loglik+2*(p+2)
    bic = -2*loglik+np.log(T-p)*(p+2)
    return arfit_eval(loglik,aic,bic)
    
def portmanteau(p,S,arfit):
    v_cent = S@arfit.w-arfit.μ
    T = len(v_cent)
    υ = np.sum(v_cent[p:]**2)/(T-p)
    port = 0
    for k in range(p):
        γ_k = 0
        lag = k+1
        for t in range(p,T-lag):
            γ_k = γ_k+v_cent[t]*v_cent[t+lag]
        γ_k = γ_k/(T-p-lag)
        port = port+(γ_k/υ)**2
    port = port/p
    return port

def crossingrate(p,S,arfit):
    v_cent = S@arfit.w-arfit.μ
    T = len(v_cent)
    χ = 0
    for t in range(p,T-1):
        if (v_cent[t]*v_cent[t+1]<=0):
            χ = χ+1
    χ = χ/(T-p-1)
    return χ

def get_windows(p,S_df,window_IS,window_OS):
    windows = 0
    idxcurr = window_IS
    while idxcurr < S_df.shape[0]:
        idxcurr = idxcurr+window_OS+p
        windows = windows+1
    return windows

def standardize(array):
    array_z = np.zeros(np.shape(array))
    for i in range(np.shape(array)[0]):
        curr = array[i,:]
        if np.std(curr) != 0:
            array_z[i,:] = (curr-np.mean(curr))/np.std(curr)
    return array_z

## Joint maximum likelihood

In [ ]:
def joint_loglik_obj_l2(params,p,S,η):
    T = S.shape[0]
    N = S.shape[1]
    c = params[0]
    ϕ = params[1:(p+1)]
    σ = params[p+1]
    w = params[(p+2):(N+p+2)]
    v = S@w
    l = 0
    for t in range(p,T):
        l = l+(v[t]-c-ϕ@np.flip(v[(t-p):t]))**2
    l = 0.5*np.log(σ*σ)+(l+η*la.norm(w,ord=2)**2)/(2*(T-p)*(σ*σ))
    return l

def get_constraints(p,N,ϕ_min,ϕ_max):
    A = np.zeros((p+3,N+p+2))
    for i in range(p+2):
        A[i,i] = 1
    A[p+2,(p+2):(N+p+2)] = 1
    lb = np.append(np.append(np.array([-np.inf]),ϕ_min),np.array([0,1]))
    ub = np.append(np.append(np.array([np.inf]),ϕ_max),np.array([np.inf,1]))
    return opt.LinearConstraint(A,lb,ub)

def mrpo_arfit_jml_l2(p,S,ϕ_min,ϕ_max,η):
    N = S.shape[1]
    params_0_cϕ = np.append(np.array([1]),np.ones(p)/(2*p))
    params_0_σw = np.append(np.array([1]),np.ones(N)/N) 
    params_0 = np.append(params_0_cϕ,params_0_σw)
    constraints = get_constraints(p,N,ϕ_min,ϕ_max)
    fit = opt.minimize(joint_loglik_obj_l2,params_0,args=(p,S,η),method='trust-constr',constraints=constraints)
    c_star = fit.x[0]
    ϕ_star = fit.x[1:(p+1)]
    σ_star = fit.x[p+1]
    w_star = fit.x[(p+2):(N+p+2)]
    l_star = joint_loglik_obj_l2(fit.x,p,S,η)
    return arfit_output(c_star,ϕ_star,σ_star,w_star,l_star)

## Closed-form expression

In [ ]:
def mrpo_arfit_cf_l2(p,S,ϕ,η):
    T = S.shape[0]
    N = S.shape[1]
    D = np.zeros((T-p,N))
    for t in range(p,T):
        for i in range(N):
            D[t-p,i] = S[t,i]-ϕ@np.flip(S[(t-p):t,i])
    DT = np.transpose(D)
    DTDinv = la.inv(DT@D+η*np.eye(N)) 
    eT = np.ones(T-p)
    eN = np.ones(N)
    x = DTDinv@DT@eT
    y = DTDinv@eN
    z = DT@eT
    α = eN@DTDinv@DT@eT
    β = eN@DTDinv@eN
    γ = eT@eT
    ϕ_star = ϕ
    c_star = (y@z)/(β*γ-β*x@z+α*y@z)
    w_star = c_star*x+((1-α*c_star)/β)*y
    σ_star = np.sqrt((la.norm(D@w_star-c_star*eT,ord=2)**2+η*la.norm(w_star)**2)/(T-p))
    θ_star_cϕ = np.append(np.array([c_star]),ϕ_star)
    θ_star_σw = np.append(np.array([σ_star]),w_star)
    θ_star = np.append(θ_star_cϕ,θ_star_σw)
    l_star = joint_loglik_obj_l2(θ_star,p,S,η)
    return arfit_output(c_star,ϕ_star,σ_star,w_star,l_star)

## Two-step maximum likelihood

In [ ]:
# here, the only parameters are φ_1,...,φ_p

def loglik_obj_l2(params,p,S,η):
    T = S.shape[0]
    N = S.shape[1]
    ϕ = params
    arfit_cf_l2 = mrpo_arfit_cf_l2(p,S,ϕ,η)
    c = arfit_cf_l2.c
    w = arfit_cf_l2.w
    σ = arfit_cf_l2.σ
    D = np.zeros((T-p,N))
    for t in range(p,T):
        for i in range(N):
            D[t-p,i] = S[t,i]-ϕ@np.flip(S[(t-p):t,i])
    eT = np.ones(T-p)
    l = 0.5*np.log(σ*σ)+(la.norm(D@w-c*eT,ord=2)**2+η*la.norm(w)**2)/(2*(T-p)*σ*σ)
    return l

def mrpo_arfit_ml_l2(p,S,ϕ_min,ϕ_max,η):
    N = S.shape[1]
    params_0 = np.ones(p)/(2*p)
    bounds = opt.Bounds(ϕ_min,ϕ_max)
    fit = opt.minimize(loglik_obj_l2,params_0,args=(p,S,η),method='L-BFGS-B',bounds=bounds)
    ϕ_star = fit.x
    arfit_cf_l2_star = mrpo_arfit_cf_l2(p,S,ϕ_star,η)
    c_star = arfit_cf_l2_star.c
    w_star = arfit_cf_l2_star.w
    σ_star = arfit_cf_l2_star.σ
    l_star = loglik_obj_l2(ϕ_star,p,S,η)
    return arfit_output(c_star,ϕ_star,σ_star,w_star,l_star)

## Run functions

In [ ]:
# # Choose parameters

# p = 1

# idx_start = 0
# window_IS = 200+p
# window_OS = 100
# S_IS = S_norm_df.to_numpy()[idx_start:(idx_start+window_IS),:]
# S_OS = S_norm_df.to_numpy()[(idx_start+window_IS-p):(idx_start+window_IS+window_OS),:]
# S = S_IS

# ϕ_min = -np.ones(p)
# ϕ_max = np.ones(p)

# η = 10

# # Joint maximum likelihood

# time_0 = time.time()
# arfit_jml_l2 = mrpo_arfit_jml_l2(p,S,ϕ_min,ϕ_max,η)
# time_1 = time.time()
# print("Joint maximum likelihood\n","Elapsed time:",time_1-time_0)

# print("c:",arfit_jml_l2.c)
# print("ϕ:",arfit_jml_l2.ϕ)
# print("σ:",arfit_jml_l2.σ)
# print("w:",arfit_jml_l2.w)
# print("μ:",arfit_jml_l2.μ)
# print("l:",arfit_jml_l2.l)
# v_IS_jml_l2 = S_IS[p:,]@arfit_jml_l2.w
# plt.plot(v_IS_jml_l2)
# plt.grid()
# plt.show()

# # Verify closed-form expressions

# ϕ = arfit_jml_l2.ϕ

# time_0 = time.time()
# arfit_cf_l2 = mrpo_arfit_cf_l2(p,S,ϕ,η)
# time_1 = time.time()
# print("Closed-form\n","Elapsed time:",time_1-time_0)

# print("c:",arfit_cf_l2.c)
# print("ϕ:",arfit_cf_l2.ϕ)
# print("σ:",arfit_cf_l2.σ)
# print("w:",arfit_cf_l2.w)
# print("μ:",arfit_cf_l2.μ)
# print("l:",arfit_cf_l2.l)
# v_IS_cf_l2 = S_IS[p:,]@arfit_cf_l2.w
# plt.plot(v_IS_cf_l2)
# plt.grid()
# plt.show()

# # Two-step maximum likelihood 

# time_0 = time.time()
# arfit_ml_l2 = mrpo_arfit_ml_l2(p,S,ϕ_min,ϕ_max,η)
# time_1 = time.time()
# print("Two-step maximum likelihood\n","Elapsed time:",time_1-time_0)

# print("c:",arfit_ml_l2.c)
# print("ϕ:",arfit_ml_l2.ϕ)
# print("σ:",arfit_ml_l2.σ)
# print("w:",arfit_ml_l2.w)
# print("μ:",arfit_ml_l2.μ)
# print("l:",arfit_ml_l2.l)
# v_IS_ml_l2 = S_IS[p:,]@arfit_ml_l2.w
# plt.plot(v_IS_ml_l2)
# plt.grid()
# plt.show()

# # Check stationarity

# print("Stationarity condition:",(np.max(np.abs(np.roots(np.append(np.array([1]),-ϕ))))<1))

## Benchmark: $\eta$ = 0

In [ ]:
# p_max = 10
# window_IS = 200+p_max
# window_OS = 50
# n_win = get_windows(p_max,S_df,window_IS,window_OS)

# jywealth_IS = np.zeros((n_win,η_max))
# jywealth_OS = np.zeros((n_win,η_max))
# jysharpe_IS = np.zeros((n_win,η_max))
# jysharpe_OS = np.zeros((n_win,η_max))

# portmanteau_IS = np.zeros((n_win,p_max))
# portmanteau_OS = np.zeros((n_win,p_max))
# crossingrate_IS = np.zeros((n_win,p_max))
# crossingrate_OS = np.zeros((n_win,p_max))

# loglik_IS = np.zeros((n_win,p_max))
# loglik_OS = np.zeros((n_win,p_max))
# aic_IS = np.zeros((n_win,p_max))
# aic_OS = np.zeros((n_win,p_max))
# bic_IS = np.zeros((n_win,p_max))
# bic_OS = np.zeros((n_win,p_max))

# for i in range(n_win):
#     for j in range(p_max):
#         time_0 = time.time()
#         η = 0
#         p = j+1
#         idx_start = i*window_OS
#         S_IS = S_df.to_numpy()[(idx_start+p_max-p):(idx_start+window_IS),:]
#         S_OS = S_df.to_numpy()[(idx_start+window_IS-p):(idx_start+window_IS+window_OS),:]
#         ϕ_min = -np.ones(p)
#         ϕ_max = np.ones(p)
#         arfit_ml_l2 = mrpo_arfit_ml_l2(p,S_IS,ϕ_min,ϕ_max,η)
#         jywealth_IS[i,j] = jywealth(S_IS,S_IS,arfit_ml_l2)
#         jywealth_OS[i,j] = jywealth(S_IS,S_OS,arfit_ml_l2)
#         jysharpe_IS[i,j] = jysharpe(S_IS,S_IS,arfit_ml_l2)
#         jysharpe_OS[i,j] = jysharpe(S_IS,S_OS,arfit_ml_l2)
#         portmanteau_IS[i,j] = portmanteau(p,S_IS,arfit_ml_l2)
#         portmanteau_OS[i,j] = portmanteau(p,S_OS,arfit_ml_l2)
#         crossingrate_IS[i,j] = crossingrate(p,S_IS,arfit_ml_l2)
#         crossingrate_OS[i,j] = crossingrate(p,S_OS,arfit_ml_l2)
#         arfit_eval_IS = likelihood(p,S_IS,arfit_ml_l2)
#         arfit_eval_OS = likelihood(p,S_OS,arfit_ml_l2)
#         loglik_IS[i,j] = arfit_eval_IS.loglik
#         loglik_OS[i,j] = arfit_eval_OS.loglik
#         aic_IS[i,j] = arfit_eval_IS.aic
#         aic_OS[i,j] = arfit_eval_OS.aic
#         bic_IS[i,j] = arfit_eval_IS.bic
#         bic_OS[i,j] = arfit_eval_OS.bic
#         time_1 = time.time()
#         print(i,j,"- elasped time -",time_1-time_0)
        
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-portmanteau-IS.npz",portmanteau_IS)
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-portmanteau-OS.npz",portmanteau_OS)
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-crossingrate-IS.npz",crossingrate_IS)
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-crossingrate-OS.npz",crossingrate_OS)
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-loglik-IS.npz",loglik_IS)
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-loglik-OS.npz",loglik_OS)
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-aic-IS.npz",aic_IS)
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-aic-OS.npz",aic_OS)
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-bic-IS.npz",bic_IS)
# np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-bench-bic-OS.npz",bic_OS)

## Fix model order, vary tuning parameter

### Set parameters

In [ ]:
η_max = 10+1
p = 1

p_max = 10
window_IS = 210+p_max
window_OS = 42
n_win = get_windows(p_max,S_df,window_IS,window_OS)

### Fit portfolios

In [ ]:
portmanteau_IS = np.zeros((n_win,η_max))
portmanteau_OS = np.zeros((n_win,η_max))
crossingrate_IS = np.zeros((n_win,η_max))
crossingrate_OS = np.zeros((n_win,η_max))

loglik_IS = np.zeros((n_win,η_max))
loglik_OS = np.zeros((n_win,η_max))
aic_IS = np.zeros((n_win,η_max))
aic_OS = np.zeros((n_win,η_max))
bic_IS = np.zeros((n_win,η_max))
bic_OS = np.zeros((n_win,η_max))

for i in range(n_win):
    for j in range(η_max):
        time_0 = time.time()
        η = j*0.01
        idx_start = i*window_OS
        S_IS = S_norm_df.to_numpy()[(idx_start+p_max-p):(idx_start+window_IS),:]
        S_OS = S_norm_df.to_numpy()[(idx_start+window_IS-p):(idx_start+window_IS+window_OS),:]
        ϕ_min = -np.ones(p)
        ϕ_max = np.ones(p)
        arfit_ml_l2 = mrpo_arfit_ml_l2(p,S_IS,ϕ_min,ϕ_max,η)
        portmanteau_IS[i,j] = portmanteau(p,S_IS,arfit_ml_l2)
        portmanteau_OS[i,j] = portmanteau(p,S_OS,arfit_ml_l2)
        crossingrate_IS[i,j] = crossingrate(p,S_IS,arfit_ml_l2)
        crossingrate_OS[i,j] = crossingrate(p,S_OS,arfit_ml_l2)
        arfit_eval_IS = likelihood(p,S_IS,arfit_ml_l2)
        arfit_eval_OS = likelihood(p,S_OS,arfit_ml_l2)
        loglik_IS[i,j] = arfit_eval_IS.loglik
        loglik_OS[i,j] = arfit_eval_OS.loglik
        aic_IS[i,j] = arfit_eval_IS.aic
        aic_OS[i,j] = arfit_eval_OS.aic
        bic_IS[i,j] = arfit_eval_IS.bic
        bic_OS[i,j] = arfit_eval_OS.bic
        time_1 = time.time()
        print(i,j,"- elasped time -",time_1-time_0)
        
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-portmanteau-IS.npz",portmanteau_IS)
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-portmanteau-OS.npz",portmanteau_OS)
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-crossingrate-IS.npz",crossingrate_IS)
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-crossingrate-OS.npz",crossingrate_OS)
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-loglik-IS.npz",loglik_IS)
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-loglik-OS.npz",loglik_OS)
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-aic-IS.npz",aic_IS)
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-aic-OS.npz",aic_OS)
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-bic-IS.npz",bic_IS)
np.savez("C:\\Users\\zhubr\\Desktop\\STAT 491\\results\\"+data_str+"l2-p"+str(p)+"-bic-OS.npz",bic_OS)

### Plot figures

In [ ]:
plt.figure(figsize=(6,6))
plt.subplot(2,1,1)
plt.title(data_str+"-l2-p"+str(p))
plt.plot(np.mean(portmanteau_IS,axis=0),marker='o',label="IS Port.")
plt.plot(np.mean(portmanteau_OS,axis=0),marker='o',label="OS Port.")
plt.ylabel("Port. Stat")
plt.legend()
plt.grid()
plt.subplot(2,1,2)
plt.plot(np.mean(crossingrate_IS,axis=0),marker='o',label="IS X-ing")
plt.plot(np.mean(crossingrate_OS,axis=0),marker='o',label="OS X-ing")
plt.ylabel("X-ing Stat")
plt.xlabel("Tuning Parameter (x0.01)")
plt.legend()
plt.grid()
plt.savefig("C:\\Users\\zhubr\\Desktop\\STAT 491\\figures\\"+data_str+"-l2-p"+str(p)+"-stats.jpg")
plt.show()

portmanteau_IS_z = standardize(portmanteau_IS)
portmanteau_OS_z = standardize(portmanteau_OS)
crossingrate_IS_z = standardize(crossingrate_IS)
crossingrate_OS_z = standardize(crossingrate_OS)
plt.figure(figsize=(6,6))
plt.subplot(2,1,1)
plt.title(data_str+"-l2-p"+str(p))
plt.plot(np.mean(portmanteau_IS_z,axis=0),marker='o',label="IS Port.")
plt.plot(np.mean(portmanteau_OS_z,axis=0),marker='o',label="OS Port.")
plt.ylabel("Z-Score")
plt.legend()
plt.grid()
plt.subplot(2,1,2)
plt.plot(np.mean(crossingrate_IS_z,axis=0),marker='o',label="IS X-ing")
plt.plot(np.mean(crossingrate_OS_z,axis=0),marker='o',label="OS X-ing")
plt.ylabel("Z-Score")
plt.xlabel("Tuning Parameter (x0.01)")
plt.legend()
plt.grid()
plt.savefig("C:\\Users\\zhubr\\Desktop\\STAT 491\\figures\\"+data_str+"-l2-p"+str(p)+"-stats-z.jpg")
plt.show()

plt.figure(figsize=(6,4))
plt.subplot(2,1,1)
plt.title(data_str+"-l2-p"+str(p))
plt.plot(np.mean(loglik_IS,axis=0),marker='o',label="IS Loglik")
plt.ylabel("Value")
plt.legend()
plt.grid()
plt.subplot(2,1,2)
plt.plot(np.mean(loglik_OS,axis=0),marker='o',label="OS Loglik",color='C1')
plt.ylabel("Value")
plt.xlabel("Tuning Parameter (x0.01)")
plt.legend()
plt.grid()
plt.savefig("C:\\Users\\zhubr\\Desktop\\STAT 491\\figures\\"+data_str+"-l2-p"+str(p)+"-loglik.jpg")
plt.show()

plt.figure(figsize=(6,6))
plt.subplot(2,1,1)
plt.title(data_str+"-l2-p"+str(p))
plt.plot(np.mean(aic_IS,axis=0),marker='o',label="IS AIC",color='C2')
plt.plot(np.mean(bic_IS,axis=0),marker='o',label="IS BIC",color='C3')
plt.ylabel("Value")
plt.legend()
plt.grid()
plt.subplot(2,1,2)
plt.plot(np.mean(aic_OS,axis=0),marker='o',label="OS AIC",color='C2')
plt.plot(np.mean(bic_OS,axis=0),marker='o',label="OS BIC",color='C3')
plt.ylabel("Value")
plt.xlabel("Tuning Parameter (x0.01)")
plt.legend()
plt.grid()
plt.savefig("C:\\Users\\zhubr\\Desktop\\STAT 491\\figures\\"+data_str+"-l2-p"+str(p)+"-aicbic.jpg")
plt.show()